# Instrukcja - Transformacja Hougha

### Cel:
- zapoznanie z transformacją Hougha dla pojedynczego punktu,
- kilku punktów, prostych figur
- wykorzystanie transformacji Hougha do detekcji linii prostych na rzeczywistym obrazie
- transformacja Hougha w przestrzeni ab

### Transformacja Hough'a

Transformacja Hougha dla prostych jest metodą detekcji współliniowych punktów. Każda prosta może być jednoznacznie przedstawiona za pomocą dwóch parametrów. Przestrzeń tych parametrów to przestrzeń Hougha. Najczęściej wykorzystywanymi parametrami w tej metodzie są współczynniki ρ,θ

opisujące równanie prostej w postaci normalnej:

ρ=x⋅cos(θ)+y⋅sin(θ)

gdzie: ρ - promień wodzący, θ - kąt pomiędzy ρ a osią OX.

Własności transformacji Hougha:
- prostej w przestrzeni kartezjańskiej odpowiada punkt w przestrzeni Hougha
- pękowi prostych przechdzących przez punkt w przestrzeni kartezjańskiej odpowiada krzywa sinusoidalna w przestrzeni Hougha
- punkty leżące na tej samej prostej (w przestrzeni kartezjańskiej) korespondują z sinusoidami przechodzącymi przez wspólny punkt w przestrzeni Hougha.

Metoda wyliczania transformacji Hougha składa się z następujących kroków:
- przez każdy badany (różny od zera) punkt obrazu prowadzony jest pęk prostych, przechodzących przez ten punkt
- każda z tych prostych transformowana jest do przestrzeni Hougha i tworzy tam punkt o współrzędnych ρ,θ
- w ten sposób, każdy punkt obrazu pierwotnego (pęk prostych) jest odwzorowany w sinusoidalną krzywą w przestrzeni Hougha

Przestrzeń Hougha jest przestrzenią akumulacyjną tzn. punkty sinusoidalnych krzywych, wygenerowanych dla punktów obrazu pierwotnego dodają się w miejscach, w których krzywe te przecinają się. Powstałe w ten sposób (w przestrzeni Hougha) maksima odpowiadają zbiorom punktów, należących do jednej prostej. Współrzędne ρ,θ
tego maksimum jednoznacznie określają położenie prostej na obrazie pierwotnym.

### Transformacja Hougha dla małej liczby punktów.
   1. Uruchom poniższy kod. W tablicy `im` wskaż jeden punkt, dla którego ma zostać obliczona transformata.

Poprzednio wysłałem Panu złą wersje sprawozdania. Od razu nie zauważyłem tego faktu, ponieważ miałem usunąć wszystkie otzymane wyniki by umieścić się w progach rozmiaru pliku. Po różnych próbach uruchamienia treści, kilkakrotnego przerobienia i poprawienia kodu w konsekwencji mogły wystąpić niezgodności nazw zmiennych lub funkcji za co bardzo przepraszam.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
import os
import copy

if not os.path.exists("kwadraty.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/kwadraty.png --no-check-certificate
if not os.path.exists("lab112.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/lab112.png --no-check-certificate
if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/dom.png --no-check-certificate

im = np.zeros((64,64), dtype=np.uint8)

im[18, 31] = 1

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
ax.imshow(im, 'gray')
ax.axis('off')


3. Wykonaj transformację Hougha obazu im. Wykorzystaj funkcję *hough_line* z modułu _skimage.transform_. Funkcja zwraca: macierz H (przestrzeń Hougha) oraz dwa wektory theta i rho dla kolejnych 
4. Wyświetl przestrzeń Hougha za pomocą funkcji _plt.imshow_ (można też wykorzystać poniższą funkcję *show_hough*). Jak "wygląda" pojedynczy punkt w przestrzeni Hougha?

In [ ]:
def show_hough(h, image):
    fig, axes = plt.subplots(1, 2, figsize=(15, 10))
    ax = axes.ravel()

    ax[0].imshow(image, 'gray')
    ax[0].set_title('Input image')
    ax[0].set_axis_off()

    ax[1].imshow(h, 'gray')
    ax[1].set_title('Hough transform')
    ax[1].set_xlabel('Angles (degrees)')
    ax[1].set_ylabel('Distance (pixels)')
    ax[1].axis('image')
    
    plt.tight_layout()
    plt.show()

In [ ]:
H, theta, ro = hough_line(im)

show_hough(H, im)

Pojedynczy punkt w przestrzeni jest reprezentowany poprzez jedną krzywą.

5. Powtórz punkty 1-4, ale tym razem klinkij dwa punkty. Jak zmienia się przestrzeń Hougha?
6. Powtórz punkty 1-4, ale tym razem kliknij kilka punktów starając się aby były współliniowe. Zaobserwuj zmiany w przestrzeni Hougha
7. Poeksperymentuj z różnymi układami punktów

In [ ]:
im2 = np.zeros((64,64), dtype=np.uint8)

im2[8, 41] = 1
im2[21, 37] = 1

H2, theta2, ro2 = hough_line(im2)
show_hough(H2, im2)

Dodanemu punktowi odpowiada druga krzywa Hougha.

In [ ]:
im3 = np.zeros((64,64), dtype=np.uint8)

im3[25, 10:55] = 1

H3, theta3, ro3 = hough_line(im3)
show_hough(H3, im3)

Prostej odpowiada obszar w przestrzeni Hougha

In [ ]:
im4 = np.zeros((64,64), dtype=np.uint8)

im4[10:55, 25] = 1

H4, theta4, ro4 = hough_line(im4)
show_hough(H4, im4)

Prostej pionowej odpowiada obszar, składający się z prostych, przecinających się w wspólnym punkcie środkowym (maksimum) w przestrzeni Hougha.

### Transformata Hougha dla pojedynczego obiektu

W tym podpunkcie pokazane zostanie praktycznie wykorzystanie transformaty Hougha - do detekcji prostych na sztucznym rysunku.

   1. Wczytaj obraz "kwadraty.png". Wyświetl go.
   2. Wykonaj detekcję krawędzi jedną z metod gradientowych. Ważne aby obraz krawędzi był jak najlepszej jakości - co oznacza cienkie (nawet niekoniecznie ciągłe) krawędzie - dla tego przypadku nie powinno być trudne do uzyskania. Wyświetl obraz po detekcji krawędzi.
   3. Wykonaj transformatę Hougha obrazu krawędziowego. Wykorzystaj funkcję *hough\_line*.
   4. Wyświetl macierz H. Czy widoczna jest taka liczba maksimów jakiej się spodziewamy?

In [ ]:
kwad = cv2.imread('kwadraty.png', cv2.IMREAD_GRAYSCALE)

plt.imshow(kwad)
plt.xticks([]), plt.yticks([])
plt.gray()
plt.show()

In [ ]:
structuring_element = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))

kwad_grad = cv2.morphologyEx(kwad, cv2.MORPH_GRADIENT, structuring_element)

plt.imshow(kwad_grad)
plt.xticks([]), plt.yticks([])
plt.gray()
plt.show()

In [ ]:
H_kwad, theta_kwad, ro_kwad = hough_line(kwad_grad)

plt.figure(figsize=(20, 20))
plt.imshow(H_kwad)
plt.xticks([]), plt.yticks([])
plt.gray()
plt.show()

Odpowiednio do 8 linii prostych z obrazu wejściowego, otrzymaliśmy 8 maksimów w przestrzenie Hougha.

 5. W module skimage.transform dostępna jest funkcja do automatycznej analizy przestrzeni Hougha - wyszukiwania maksimów - *hough\_line\_peaks*. Jako parametry przyjmuje ona wyniki funkcji *hough\_line* (macierz H, theta i rho). Dodatkowo można podać próg powyżej którego punkt uznawany jest za maksimum (_threshold_ - domyslnie jest to połowa maksimum w przestrzeni H) oraz liczbę poszukiwanych maksimów (*num_peaks*). Funkcja zwraca współrzędne maksimów. Wykorzystaj funkcję *hough\_line\_peaks* do znalezienia maksimów odpowiadających krawędziom kwadratów.
 6. Wyświetl macierz H używając konstrukcji:

In [ ]:
peaks_kwad = hough_line_peaks(H_kwad, theta_kwad, ro_kwad)

In [ ]:
fig,ax = plt.subplots(1)
ax.set_aspect('equal')

ax.imshow(peaks_kwad, 'gray') 
plt.plot()


Taki zapis pozwoli na dołożenie annotacji (okręgów) w miejscach znalezionych maksimów. Wyrysowanie okręgu w punkcie x, y (o rozmiarze 10, w czerwonym kolorze, bez wypełnienia środka) realizuje wywołanie: 

**circle = plt.Circle((x, y), 10, color='r', fill=False)**

natomiast dołożenie takiego okręgu do obrazu to:

**ax.add_patch(circle)**

Zaznacz maksima na obrazie wykorzystując rezultat funkcji *hough\_line\_peaks* biorąc pod uwagę, że zwraca ona kąty w radianach z przedziału od -pi/2 do pi/2, a rho z przedziału od -r/2 do r/2 gdzie r to pionowy rozmiar przestrzeni Hougha. 

In [ ]:
def show_hough_line_peaks(H, theta, ro):
    peaks = hough_line_peaks(H, theta, ro)
    
    fig,ax = plt.subplots(1)
    fig.set_size_inches(8, 20)

    ax.set_aspect('equal')
    ax.imshow(H, 'gray')
    plt.plot()

    heigth_hk, width_hk = np.shape(H)

    for i, x, y in zip(*peaks):
        circle = plt.Circle(((x*60)+90, y+heigth_hk/2 ), 10, color='b', fill=False)
        ax.add_patch(circle)   
    

In [ ]:
show_hough_line_peaks(H_kwad, theta_kwad, ro_kwad)

7. Istnieje też możliwość przeprowadzenia transformacji Hougha z użyciem biblioteki OpenCV. W bibliotece znajdują się dwie wersje funkcji wyszukującej linie proste - 'klasyczna' - _HoughLines_ oraz probabilistyczna _HoughLinesP_. Zadna z nich nie zwraca przestrzeni Hougha. Wynikiem działania pierwszej jest lista parametrów prostych (krotki zawierające rho, theta). Druga zwraca krotki 4-ro elementowe ze współrzędnymi końców odcinków wykorzystanych do wylicznia parametrów (czyli znalezienia prostej). 
8. Wyznacz linie obecne na obrazie za pomocą funkcji _HoughLines_. Wykryte linie wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). Do wyświetlania linii wykorzystaj przykładowy kod:

In [ ]:
def draw_lines(img, houghLines, color = [0, 255, 0], thickness=1):
    for line in zip(*houghLines):
        for rho,theta in line:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
 
            cv2.line(img,(x1,y1),(x2,y2),color,thickness)   

In [ ]:
kwad_rgb = cv2.imread('kwadraty.png')
blurred_image = cv2.GaussianBlur(kwad, (3, 3), 0)
edges_image = cv2.Canny(blurred_image, 50, 120)
 
rho_resolution = 1
theta_resolution = np.pi/180
threshold = 55
 
hough_lines = cv2.HoughLines(edges_image, rho_resolution , theta_resolution , threshold)

hough_lines_image = np.zeros_like(kwad_rgb)
draw_lines(hough_lines_image, hough_lines)
original_image_detected = cv2.addWeighted(kwad_rgb, 0.9, hough_lines_image, 0.8, 0)

 

fig, axs = plt.subplots(1, 2)
    
fig.set_size_inches(15, 10)
axs[0].imshow(kwad, 'gray')
axs[0].axis('off')
axs[0].set_title('Obraz oryginalny',)
    
axs[1].imshow(original_image_detected)
axs[1].axis('off')
axs[1].set_title('Obraz z wykrytymi liniami')
    
plt.plot()

Wyświetlanie linii nie jest ograniczone kształtami obiektów i przechodzi przez całą przestrzeń.

9. Wyznacz odcinki obecne na obrazie za pomocą funkcji _HoughLinesP_. Wykryte odcinki wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). 

In [ ]:
kwad_rgb = cv2.imread('kwadraty.png')
original_image_detected_prob = copy.copy(kwad_rgb) 
blurred_image = cv2.GaussianBlur(kwad, (3, 3), 0)
edges_image = cv2.Canny(blurred_image, 50, 120)
 


hough_linesP = cv2.HoughLinesP(edges_image, rho = 1, theta = 1*np.pi/180, threshold = 45, minLineLength = 20, maxLineGap = 50)

line_color = [0, 255, 0] 
line_thickness = 2
for i in range(hough_linesP.shape[0]):
    x1 = hough_linesP[i][0][0]
    y1 = hough_linesP[i][0][1]    
    x2 = hough_linesP[i][0][2]
    y2 = hough_linesP[i][0][3]    
    cv2.line(original_image_detected_prob,(x1,y1), (x2,y2), line_color, line_thickness)

    
    
fig, axs = plt.subplots(1, 2)
    
fig.set_size_inches(15, 10)
axs[0].imshow(kwad_rgb, 'gray')
axs[0].axis('off')
axs[0].set_title('Obraz oryginalny',)
    
axs[1].imshow(original_image_detected_prob)
axs[1].axis('off')
axs[1].set_title('Obraz z wykrytymi liniami')
   
plt.plot()    
    


### Transformata Hougha dla obrazu rzeczywistego.

Bazując na kodzie stworzonym w punkcie B wyszukamy linie na obrazie rzeczywistym.
   1. Wczytaj obraz "lab112.png". Wyświetl go.
   2. Wykorzystując wszystkie poznane techniki przetwarzania obrazów (filtracja, przekształcenia morfologiczne, binaryzację, detekcję krawędzi) wyodrębnij krawędzie samych kwadratów - tak aby były jak najlepszej jakości (cienkie) - jednocześnie eliminując z obrazu zakłócenia.
   3. Wykorzystaj funkcje *hough_line* i *hough_line_peaks* do detekcji linii na obrazie, a następnie np. wykorzystując kod z punktu 8 poprzedniego ustępu wyrysuj na oryginalnym obrazie znalezione linie.

In [ ]:
lab = cv2.imread('lab112.png')
lab_grey = cv2.imread('lab112.png', cv2.IMREAD_GRAYSCALE)

plt.imshow(lab)
plt.xticks([]), plt.yticks([])
plt.gray()
plt.show()



In [ ]:
blurred_lab = cv2.GaussianBlur(lab_grey, (5, 5), 0)
lab_canny = cv2.Canny(blurred_lab, 100, 200)

plt.imshow(lab_canny)
plt.xticks([]), plt.yticks([])
plt.gray()
plt.show()

In [ ]:
H_lab, theta_lab, ro_lab = hough_line(lab_canny)
peaks_lab = hough_line_peaks(H_lab, theta_lab, ro_lab)

i, x, y = peaks_lab
a = y.tolist()
b = x.tolist()
coords = np.array([[a],[b]]).T

show_hough_line_peaks(H_lab, theta_lab, ro_lab)

hough_lines_lab_img = np.zeros_like(lab)
draw_lines(hough_lines_lab_img, coords, thickness=2)
original_image_lab_detected = cv2.addWeighted(lab, 1, hough_lines_lab_img, 0.5, 0)

 
fig, axs = plt.subplots(1, 2)
    
fig.set_size_inches(15, 10)
axs[0].imshow(lab)
axs[0].axis('off')
axs[0].set_title('Obraz oryginalny')
   
axs[1].imshow(original_image_lab_detected)
axs[1].axis('off')
axs[1].set_title('Obraz z wykrytymi liniami')
   
plt.plot()

4. Wczytaj obraz "dom.png". Wypróbuj działanie transformacji Hougha na tym obrazie z wykorzystaniem funkcji _cv2.HoughLinesP_  (oczywiście po odpowiednich przekształceniach). Postaraj się tak przygotować obraz z krawędziami i dobrać parametry aby wyrysować na oryginalnym obrazie odcinki obejmujące zarysy domu. Weź pod uwage dodatkowe parametry funkcji, takie jak:   minLineLength, maxLineGap.

In [ ]:
house = cv2.imread('dom.png')
house_grey = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

original_image_detected_prob = copy.copy(house)
blurred_image = cv2.GaussianBlur(house_grey, (5, 5), 0)
edges_image = cv2.Canny(blurred_image, 100, 200)
 


hough_linesP = cv2.HoughLinesP(edges_image, rho = 1, theta = 1*np.pi/180, threshold = 10, minLineLength = 2, maxLineGap = 5)

line_color = [0, 255, 0] 
line_thickness = 2
for i in range(hough_linesP.shape[0]):
    x1 = hough_linesP[i][0][0]
    y1 = hough_linesP[i][0][1]    
    x2 = hough_linesP[i][0][2]
    y2 = hough_linesP[i][0][3]    
    cv2.line(original_image_detected_prob,(x1,y1), (x2,y2), line_color, line_thickness)

    
    
fig, axs = plt.subplots(1, 2)
   
fig.set_size_inches(15, 10)
axs[0].imshow(house, 'gray')
axs[0].axis('off')
axs[0].set_title('Obraz oryginalny',)
    
axs[1].imshow(original_image_detected_prob)
axs[1].axis('off')
axs[1].set_title('Obraz z wykrytymi liniami')
    
plt.plot()    